In [4]:
import pandas as pd


In [5]:
df = pd.read_csv("../datasets/nrel_training_data.csv")
df["isTurbine"] = df["isTurbine"].astype(int)
X = df[[
    "fraction_of_usable_area",
    "capacity",
    "wind_speed",
    "capacity_factor"
]]
y = df['isTurbine']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [8]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss'
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [21]:
from sklearn.metrics import confusion_matrix, classification_report

preds = model.predict(X_test)

cm = confusion_matrix(y_test, preds)
print("Confusion Matrix:")
print()
print(cm)
print()

print(classification_report(y_test, preds, digits=3))

Confusion Matrix:

[[19971  3827]
 [ 5106  9268]]

              precision    recall  f1-score   support

           0      0.796     0.839     0.817     23798
           1      0.708     0.645     0.675     14374

    accuracy                          0.766     38172
   macro avg      0.752     0.742     0.746     38172
weighted avg      0.763     0.766     0.764     38172

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x000001A677CA5EE0> (for post_run_cell), with arguments args (<ExecutionResult object at 1a67e426890, execution_count=21 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1a66d813930, raw_cell="from sklearn.metrics import confusion_matrix, clas.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/bemne/Workspace/AI4ALL-Project/xgboost/xgboost.ipynb#W5sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AttributeError: module 'matplotlib' has no attribute 'backends'

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib
import json

# Compute probabilities for ROC-AUC
y_proba = model.predict_proba(X_test)[:, 1]

# Print ROC-AUC score
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC Score: {roc_auc:.3f}")

# Print ROC curve values
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print("FPR:", fpr)
print("TPR:", tpr)

# Save the model as a .pkl file
joblib.dump(model, "xgboost_model.pkl")

# Save model config/specs as a JSON
model_specs = {
    "n_estimators": 300,
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "objective": "binary:logistic",
    "eval_metric": "logloss"
}
with open("xgboost_model_specs.json", "w") as f:
    json.dump(model_specs, f, indent=4)

# Save the model metrics in the same format as random_forest.ipynb
metrics = {
    "model": "xgboost",
    "roc_auc": roc_auc,
    "classification_report": classification_report(y_test, preds, digits=3, output_dict=True)
}
with open("xgboost_model_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)







ROC-AUC Score: 0.8473
Model saved to xgboost_wind_model.pkl
Metrics saved to xgboost_model_metrics.json
